In [1]:
# data manipulation
import pandas as pd
import numpy as np
import scipy.io.arff

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

# data pre-processing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier
import re

%matplotlib inline

In [2]:
# carregando o dataset
data, meta = scipy.io.arff.loadarff(f'data/speeddating.arff')
df = pd.DataFrame(data)

# Convertendo byte-strings para strings
str_df = df.select_dtypes([object]).stack().str.decode('utf-8').unstack()
for col in str_df:
    df[col] = str_df[col]

# ajeitar o tipo dos dados
nominal_cols = [col for col, dtype in zip(meta.names(), meta.types()) if dtype == 'nominal']
for col in nominal_cols:
    df[col] = df[col].astype('category')

## Inconsistências

In [3]:
df['met'].value_counts()

,count
met,
0.0,7644
1.0,351
7.0,3
5.0,2
3.0,1
8.0,1
6.0,1


- Temos inconsistências em gaming e reading, pois essas variáveis deveriam ter valores até 10, e estão com alguns valores superiores.
- Podemos observar também que na variável met, temos vários valores diferentes de 1 ou 0 (deveria ser uma coluna binária, para responder se a pessoa já encontrou o seu parceiro antes ou não), o que não faz sentido. como são pouquíssimas instâncias (8 de um total de quase 8000), podemos apenas removê-las.

In [4]:
# lidar com inconsistências relacionadas aos interesses
def limit_interests_above_10(df):
    df_copy = df.copy()
    interest_cols = [
        'sports', 'tvsports', 'exercise', 'dining', 'museums',
        'art', 'hiking', 'gaming', 'clubbing', 'reading',
        'tv', 'theater', 'movies', 'concerts', 'music',
        'shopping', 'yoga'
    ]
    df_copy[interest_cols] = df_copy[interest_cols].applymap(lambda x: min(x, 10))
    return df_copy

def remove_interests_above_10(df):
    df_copy = df.copy()
    interest_cols = [
        'sports', 'tvsports', 'exercise', 'dining', 'museums',
        'art', 'hiking', 'gaming', 'clubbing', 'reading',
        'tv', 'theater', 'movies', 'concerts', 'music',
        'shopping', 'yoga'
    ]

    df_copy = df_copy[df_copy[interest_cols] <= 10]
    return df_copy

In [5]:
df_test = df.copy()
df_test = limit_interests_above_10(df_test)

<ipython-input-4-c7ae1027e386>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_copy[interest_cols] = df_copy[interest_cols].applymap(lambda x: min(x, 10))


In [6]:
def limit_met_1(df):
    df_copy = df.copy()
    cols = ['met']
    df_copy[cols] = df_copy[cols].applymap(lambda x: min(x, 1))
    return df_copy

def remove_met_over_1(df):
    df_copy = df.copy()
    cols = ['met']
    df_copy = df_copy[df_copy[cols] <= 1]
    return df_copy

In [7]:
# lidar com inconsistências relacionadas à prof_o_ambitious
def limit_pref_o_ambitious_30(df):
    df_copy = df.copy()
    ambitious_cols = ['ambtition_important', 'pref_o_ambitious']
    df_copy[ambitious_cols] = df_copy[ambitious_cols].applymap(lambda x: min(x, 30))
    return df_copy

def remove_pref_o_ambitious_over_30(df):
    df_copy = df.copy()
    ambitious_cols = ['ambtition_important', 'pref_o_ambitious']
    df_copy = df_copy[df_copy[ambitious_cols] <= 30]
    return df_copy

In [8]:
df_test = limit_met_1(df_test)

<ipython-input-6-bebf02e768fa>:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_copy[cols] = df_copy[cols].applymap(lambda x: min(x, 1))


Outra inconsistência perecebida é que alguns valores categóricos que representam a mesma categoria, estão escritas de maneira diferente (como Business e business), o que leva a pensar que isso talvez aconteça com outros valores textuais também, então aplicaremos uma transformação para garantir que todos os valores categóricos estejam minúsculos

In [9]:
def set_lower(df):
    df_copy = df.copy()
    df_copy = df.map(lambda x: x.lower() if isinstance(x, str) else x)
    return df_copy

In [10]:
df_test = set_lower(df_test)

In [11]:
def replace_invalid_nan(df):
    df_copy = df.copy()
    df_copy.replace("?", np.nan, inplace=True)
    return df_copy

In [12]:
df_test = replace_invalid_nan(df_test)

<ipython-input-11-c176750a2b45>:3: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_copy.replace("?", np.nan, inplace=True)


## Processamento de dados ausentes

In [13]:
float_nan_cols = [
    "pref_o_attractive", "pref_o_sincere",
    "pref_o_intelligence", "pref_o_funny", "pref_o_ambitious",
    "pref_o_shared_interests", "interests_correlate"
]

In [14]:
def evaluate_correlation_change(original_corr, imputed_df):
    """Calculate the change in correlation matrices."""
    new_corr = imputed_df.select_dtypes(include=[np.number]).corr()

    # Ensure we only calculate over valid overlapping indices
    common_cols = original_corr.index.intersection(new_corr.index)
    return np.sum(np.abs(original_corr.loc[common_cols, common_cols] - new_corr.loc[common_cols, common_cols]).values)

In [15]:
def best_imputation(df, methods=['mean', 'median', 'most_frequent', 'knn']):
    """
    Finds the best imputation method per column by selecting the one
    that causes the least correlation change.
    """
    original_corr = df.corr()
    best_methods = {}
    best_imputed_df = df.copy()

    for col in df.columns:
        if df[col].isna().sum() > 0:  # Only process columns with missing values
            best_score = float('inf')
            best_imputed_col = None
            best_method = None

            for method in methods:
                df_temp = best_imputed_df.copy()

                if method == 'knn':
                    imputer = KNNImputer(n_neighbors=5)
                else:
                    imputer = SimpleImputer(strategy=method)

                df_temp[[col]] = imputer.fit_transform(df_temp[[col]])

                score = evaluate_correlation_change(original_corr, df_temp)

                if score < best_score:
                    best_score = score
                    best_imputed_col = df_temp[col]
                    best_method = method

            best_methods[col] = best_method
            best_imputed_df[col] = best_imputed_col

    return best_imputed_df, best_methods

In [16]:
df_selected = df_test[float_nan_cols].copy()

num_imputed_df, chosen_methods = best_imputation(df_selected)
for col in float_nan_cols:
    df_test[col] = num_imputed_df[col]

In [17]:
categ_nan_cols = ["age", "age_o", "race", "race_o", "importance_same_race", "importance_same_religion",
          "field", "attractive_o", "sinsere_o", "intelligence_o", "funny_o", "ambitous_o",
          "shared_interests_o", "attractive_important", "sincere_important", "intellicence_important",
          "funny_important", "ambtition_important", "shared_interests_important", "attractive", "sincere",
          "intelligence", "funny", "ambition", "attractive_partner", "sincere_partner", "intelligence_partner",
          "funny_partner", "ambition_partner", "shared_interests_partner", "sports", "tvsports", "exercise",
          "dining", "museums", "art", "hiking", "gaming", "clubbing", "reading", "tv", "theater", "movies",
          "concerts", "music", "shopping", "yoga", "expected_happy_with_sd_people",
          "expected_num_interested_in_me", "expected_num_matches", "like", "guess_prob_liked", "met"]

In [18]:
def input_mode(df):
    df_copy = df.copy()
    categ_nan_cols = ["age", "age_o", "race", "race_o", "importance_same_race", "importance_same_religion",
            "field", "attractive_o", "sinsere_o", "intelligence_o", "funny_o", "ambitous_o",
            "shared_interests_o", "attractive_important", "sincere_important", "intellicence_important",
            "funny_important", "ambtition_important", "shared_interests_important", "attractive", "sincere",
            "intelligence", "funny", "ambition", "attractive_partner", "sincere_partner", "intelligence_partner",
            "funny_partner", "ambition_partner", "shared_interests_partner", "sports", "tvsports", "exercise",
            "dining", "museums", "art", "hiking", "gaming", "clubbing", "reading", "tv", "theater", "movies",
            "concerts", "music", "shopping", "yoga", "expected_happy_with_sd_people",
            "expected_num_interested_in_me", "expected_num_matches", "like", "guess_prob_liked", "met"]

    for col in categ_nan_cols:
        df_copy[col] = df_copy[col].fillna(df_copy[col].mode()[0])

    return df_copy

In [19]:
temp_df = input_mode(df_test[categ_nan_cols])
for col in temp_df.columns:
    df_test[col] = temp_df[col]

df_test.isna().sum().sum()

0

## Features Categóricas

In [20]:
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
print(categorical_cols)

Index(['has_null', 'gender', 'd_d_age', 'race', 'race_o', 'samerace',
       'd_importance_same_race', 'd_importance_same_religion', 'field',
       'd_pref_o_attractive', 'd_pref_o_sincere', 'd_pref_o_intelligence',
       'd_pref_o_funny', 'd_pref_o_ambitious', 'd_pref_o_shared_interests',
       'd_attractive_o', 'd_sinsere_o', 'd_intelligence_o', 'd_funny_o',
       'd_ambitous_o', 'd_shared_interests_o', 'd_attractive_important',
       'd_sincere_important', 'd_intellicence_important', 'd_funny_important',
       'd_ambtition_important', 'd_shared_interests_important', 'd_attractive',
       'd_sincere', 'd_intelligence', 'd_funny', 'd_ambition',
       'd_attractive_partner', 'd_sincere_partner', 'd_intelligence_partner',
       'd_funny_partner', 'd_ambition_partner', 'd_shared_interests_partner',
       'd_sports', 'd_tvsports', 'd_exercise', 'd_dining', 'd_museums',
       'd_art', 'd_hiking', 'd_gaming', 'd_clubbing', 'd_reading', 'd_tv',
       'd_theater', 'd_movies', 'd_c

In [22]:
categorical_cols = df_test.select_dtypes(include=['object', 'category']).columns

In [23]:
le = LabelEncoder()

df_test["gender"] = le.fit_transform(df_test["gender"])

In [24]:
# One-Hot Encoding para 'race' e 'race_o'
df_test = pd.get_dummies(df_test, columns=["race", "race_o"], drop_first=True)

In [25]:
def convert_range_to_mean(value):
    """
    Converte uma string representando uma faixa numérica (ex: '[0-5]') para a média dos valores dentro da faixa.
    Se o valor já for um número, retorna como float.
    """
    if isinstance(value, str) and re.match(r"\[\-?\d+(\.\d+)?\s*-\s*\-?\d+(\.\d+)?\]", value):
        # Extrair os números da faixa
        numbers = [float(n) for n in re.findall(r"-?\d+\.?\d*", value)]
        return sum(numbers) / len(numbers)  # Retorna a média
    else:
        try:
            return float(value)  # Retorna o valor como float se já for numérico
        except ValueError:
            return value  # Retorna como está se não puder ser convertido

In [26]:
# colunas que começam com "d_"
categorical_numerical_cols = [col for col in df_test.columns if col.startswith("d_")]

# conversão automatica
df_test[categorical_numerical_cols] = df_test[categorical_numerical_cols].applymap(convert_range_to_mean)

# Forçar conversão para float para garantir que todas estão no formato correto
df_test[categorical_numerical_cols] = df_test[categorical_numerical_cols].astype(float)

<ipython-input-26-a4a6504821b9>:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_test[categorical_numerical_cols] = df_test[categorical_numerical_cols].applymap(convert_range_to_mean)


In [27]:
def categorize_field(field):
    if "engineering" in field.lower():
        return "Engineering"
    elif "science" in field.lower():
        return "Science"
    elif "business" in field.lower():
        return "Business"
    elif "art" in field.lower():
        return "Arts"
    else:
        return "Other"

df_test["field_grouped"] = df_test["field"].apply(lambda x: categorize_field(x))

In [28]:
# One-Hot Encoding para a nova coluna criada
df_test = df_test.drop(columns=["field"])  # remove a original que possui mais de 200 categorias
df_test = pd.get_dummies(df_test, columns=["field_grouped"], drop_first=True)

In [29]:
# converter para int, garantindo que são numéricas (já estao em formato numérico (0/1))
df_test["has_null"] = df_test["has_null"].astype(int)
df_test["samerace"] = df_test["samerace"].astype(int)
df_test["decision"] = df_test["decision"].astype(int)
df_test["decision_o"] = df_test["decision_o"].astype(int)
df_test["match"] = df_test["match"].astype(int)

# verifica se ainda tem valores categóricos
categorical_remaining = df_test.select_dtypes(include=['object', 'category']).columns
print("Features categóricas restantes:", categorical_remaining)

Features categóricas restantes: Index([], dtype='object')


## Scaling

In [30]:
def MinMaxScaling(df):
    scaler = MinMaxScaler()
    df = scaler.fit_transform(df)
    return df

In [31]:
def StandardScaling(df):
    scaler = StandardScaler()
    df= scaler.fit_transform(df)
    return df

In [32]:
df_scaled = MinMaxScaling(df_test)

In [33]:
pd.set_option('display.max_columns', None)
df_test.head(10)

,has_null,wave,gender,age,age_o,d_age,d_d_age,samerace,importance_same_race,importance_same_religion,d_importance_same_race,d_importance_same_religion,pref_o_attractive,pref_o_sincere,pref_o_intelligence,pref_o_funny,pref_o_ambitious,pref_o_shared_interests,d_pref_o_attractive,d_pref_o_sincere,d_pref_o_intelligence,d_pref_o_funny,d_pref_o_ambitious,d_pref_o_shared_interests,attractive_o,sinsere_o,intelligence_o,funny_o,ambitous_o,shared_interests_o,d_attractive_o,d_sinsere_o,d_intelligence_o,d_funny_o,d_ambitous_o,d_shared_interests_o,attractive_important,sincere_important,intellicence_important,funny_important,ambtition_important,shared_interests_important,d_attractive_important,d_sincere_important,d_intellicence_important,d_funny_important,d_ambtition_important,d_shared_interests_important,attractive,sincere,intelligence,funny,ambition,d_attractive,d_sincere,d_intelligence,d_funny,d_ambition,attractive_partner,sincere_partner,intelligence_partner,funny_partner,ambition_partner,shared_interests_partner,d_attractive_partner,d_sincere_partner,d_intelligence_partner,d_funny_partner,d_ambition_partner,d_shared_interests_partner,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,d_sports,d_tvsports,d_exercise,d_dining,d_museums,d_art,d_hiking,d_gaming,d_clubbing,d_reading,d_tv,d_theater,d_movies,d_concerts,d_music,d_shopping,d_yoga,interests_correlate,d_interests_correlate,expected_happy_with_sd_people,expected_num_interested_in_me,expected_num_matches,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,like,guess_prob_liked,d_like,d_guess_prob_liked,met,decision,decision_o,match,race_black/african american,race_european/caucasian-american,race_latino/hispanic american,race_other,race_o_black/african american,race_o_european/caucasian-american,race_o_latino/hispanic american,race_o_other,field_grouped_Business,field_grouped_Engineering,field_grouped_Other,field_grouped_Science
0,0,1.0,0,21.0,27.0,6.0,-1.0,0,2.0,4.0,-1.5,-1.5,35.00,20.00,20.00,20.00,0.00,5.00,-39.5,-2.0,-2.0,-2.0,-7.5,-7.5,6.0,8.0,8.0,8.0,8.0,6.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,15.0,20.0,20.0,15.0,15.0,15.0,-7.5,-2.0,-2.0,-7.5,-7.5,-7.5,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,6.0,9.0,7.0,7.0,6.0,5.0,-1.0,-0.5,-1.0,-1.0,-1.0,-2.5,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,-0.5,-2.5,-1.0,-0.5,-2.5,-2.5,-2.5,-2.5,-2.5,-1.0,-0.5,-2.5,-0.5,-0.5,-0.5,-1.0,-2.5,0.14,-0.165,3.0,2.0,4.0,-2.0,-1.5,-1.0,7.0,6.0,-1.0,-0.5,0.0,1,0,0,False,False,False,False,False,True,False,False,False,False,True,False
1,0,1.0,0,21.0,22.0,1.0,-0.5,0,2.0,4.0,-1.5,-1.5,60.00,0.00,0.00,40.00,0.00,0.00,-39.5,-7.5,-7.5,-39.5,-7.5,-7.5,7.0,8.0,10.0,7.0,7.0,5.0,-1.0,-1.0,-0.5,-1.0,-1.0,-2.5,15.0,20.0,20.0,15.0,15.0,15.0,-7.5,-2.0,-2.0,-7.5,-7.5,-7.5,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,7.0,8.0,7.0,8.0,5.0,6.0,-1.0,-1.0,-1.0,-1.0,-2.5,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,-0.5,-2.5,-1.0,-0.5,-2.5,-2.5,-2.5,-2.5,-2.5,-1.0,-0.5,-2.5,-0.5,-0.5,-0.5,-1.0,-2.5,0.54,-0.335,3.0,2.0,4.0,-2.0,-1.5,-1.0,7.0,5.0,-1.0,-0.5,1.0,1,0,0,False,False,False,False,False,True,False,False,False,False,True,False
2,1,1.0,0,21.0,22.0,1.0,-0.5,1,2.0,4.0,-1.5,-1.5,19.00,18.00,19.00,18.00,14.00,12.00,-2.0,-2.0,-2.0,-2.0,-7.5,-7.5,10.0,10.0,10.0,10.0,10.0,10.0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,15.0,20.0,20.0,15.0,15.0,15.0,-7.5,-2.0,-2.0,-7.5,-7.5,-7.5,6.0,8.0,8.0,8.0,7.0,-1.0,-1.0,-1.0,-1.0,-1.0,5.0,8.0,9.0,8.0,5.0,7.0,-2.5,-1.0,-0.5,-1.0,-2.5,-1.0,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,-0.5,-2.5,-1.0,-0.5,-2.5,-2.5,-2.5,-2.5,-2.5,-1.0,-0.5,-2.5,-0.5,-0.5,-0.5,-1.0,-2.5,0.16,-0.165,3.0,2.0,4.0,-2.0,-1.5,-1.0,7.0,5.0,-1.0,-2.0,1.0,1,1,1,False,False,False,False,False,False,False,False,False,False,True,False
3,0,1.0,0,21.0,23.0,2.0,-0.5,0,2.0,4.0,-1.5,-1.5,30.00,5.00,15.00,40.00,5.00,5.00,-39.5,-7.5,-7.5,-39.5,-7.5,-7.5,7.0,8.0,9